In [1]:
import pandas as pd
import bq_helper
from bq_helper import BigQueryHelper
from tqdm.auto import tqdm

C:\Users\Grazia\anaconda3\lib\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.8) or chardet (5.1.0) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

## Repositories with criterias estabilished at first

In [3]:
repositories_final = pd.read_csv("../Repositories/repositories_final.csv")

In [4]:
repositories_final

,name
0,google/brotli
1,gentilkiwi/mimikatz
2,octalmage/robotjs
3,openresty/openresty
4,jedisct1/libsodium
...,...
128,notepad-plus-plus/notepad-plus-plus
129,apple/swift
130,pytorch/pytorch
131,onnx/onnx


In [7]:
posts = pd.read_csv("../1-Vulnerability_Detection/Results/results_static_analysis_v2.csv")
posts = posts.fillna('')

In [8]:
posts

,id_post,cwe,file,line,severity,code,message
0,30877241,398,0.cpp,3,style,struct dirent *dir; ^,The scope of the variable 'dir' can be reduced.
1,40233070,0,1.cpp,27,warning,char *buf = alloca(*bufsize + 128); ...,Obsolete function 'alloca' called.
2,40233070,0,1.cpp,46,warning,char *claim_the_space = alloca(bufsize + 2...,Obsolete function 'alloca' called.
3,20898874,398,10.cpp,11,style,data = (struct snail_thread *) ptr; ...,C-style pointer casting
4,29069387,758,100.cpp,60,error,os<<input.display(&os); ^,Found a exit path from function with non-void ...
...,...,...,...,...,...,...,...
388643,47004715,457,99996.cpp,25,warning,"printf(""%d"", total); ^",Uninitialized variable
388644,47004715,457,99996.cpp,21,error,total++; ^,Uninitialized variable
388645,34116859,0,99997.cpp,,,,
388646,55820831,0,99998.cpp,,,,


In [9]:
posts_vulnerable = posts.loc[posts['severity']!='']

In [10]:
posts_vulnerable

,id_post,cwe,file,line,severity,code,message
0,30877241,398,0.cpp,3,style,struct dirent *dir; ^,The scope of the variable 'dir' can be reduced.
1,40233070,0,1.cpp,27,warning,char *buf = alloca(*bufsize + 128); ...,Obsolete function 'alloca' called.
2,40233070,0,1.cpp,46,warning,char *claim_the_space = alloca(bufsize + 2...,Obsolete function 'alloca' called.
3,20898874,398,10.cpp,11,style,data = (struct snail_thread *) ptr; ...,C-style pointer casting
4,29069387,758,100.cpp,60,error,os<<input.display(&os); ^,Found a exit path from function with non-void ...
...,...,...,...,...,...,...,...
388639,59436318,563,99994.cpp,4,style,"int i = 0, j = 0; ^",Variable 'j' is assigned a value that is never...
388641,47004715,398,99996.cpp,4,style,"int a, b, sum, c, i, total, d; ^",The scope of the variable 'sum' can be reduced.
388642,47004715,398,99996.cpp,4,style,"int a, b, sum, c, i, total, d; ^",The scope of the variable 'c' can be reduced.
388643,47004715,457,99996.cpp,25,warning,"printf(""%d"", total); ^",Uninitialized variable


## Get repositories using vulnerable code snippets

In [11]:
repositories_final_query = ', '.join(f'"{w}"' for w in repositories_final['name'])

In [12]:
repositories_final_query

'"google/brotli", "gentilkiwi/mimikatz", "octalmage/robotjs", "openresty/openresty", "jedisct1/libsodium", "spacehuhntech/esp8266_deauther", "haiwen/seafile", "tmux/tmux", "pbatard/rufus", "robertdavidgraham/masscan", "koekeishiya/yabai", "iovisor/bcc", "jarun/nnn", "swaywm/sway", "cloudwu/skynet", "genymobile/scrcpy", "jonas/tig", "hashcat/hashcat", "ventoy/ventoy", "mpv-player/mpv", "facebook/zstd", "h2o/h2o", "ish-app/ish", "acidanthera/opencorepkg", "lvgl/lvgl", "radareorg/radare2", "micropython/micropython", "timescale/timescaledb", "handbrake/handbrake", "netdata/netdata", "ffmpeg/ffmpeg", "taosdata/tdengine", "raysan5/raylib", "redis/redis", "obsproject/obs-studio", "curl/curl", "arendst/tasmota", "foundry376/mailspring", "nvidia/open-gpu-kernel-modules", "google/tink", "lottie-react-native/lottie-react-native", "tencent/matrix", "topjohnwu/magisk", "microsoft/cntk", "zhongyang219/trafficmonitor", "trojan-gfw/trojan", "bvlc/caffe", "apple/turicreate", "kyleneideck/backgroundmusi

In [13]:
ids = pd.unique(posts_vulnerable['id_post'])
print(len(ids))
chunks = list(chunks(ids, 1000))

116578


In [14]:
chunk_strings = []

for index in range(len(chunks)):
    chunk_strings.append(','.join(map(str, chunks[index])))

In [15]:
len(chunk_strings)

117

In [3]:
sotorrent = bq_helper.BigQueryHelper("sotorrent-org", "2020_12_31")

In [ ]:
results_found = []

for chunk_string in tqdm(chunk_strings):
    query = 'SELECT reference.Repo,reference.branch, reference.Path, reference.PostId, reference.SOUrl, reference.GHUrl FROM sotorrent-org.2020_12_31.PostReferenceGH reference WHERE reference.PostId IN ('+chunk_string+') and reference.Repo IN ('+repositories_final_query+')'
    result = sotorrent.query_to_pandas_safe(query, max_gb_scanned = 70)

    if not len(result)==0:
        print("Found "+str(len(result))+" data")
        results_found.append(result)

In [27]:
len(results_found)

2

In [23]:
results_found[0]

,Repo,branch,Path,PostId,SOUrl,GHUrl
0,bitcoin/bitcoin,master,src/qt/guiutil.h,21646467,https://stackoverflow.com/q/21646467,https://raw.githubusercontent.com/bitcoin/bitc...


In [25]:
results_found[1]

,Repo,branch,Path,PostId,SOUrl,GHUrl
0,tensorflow/tensorflow,master,tensorflow/core/framework/variant_op_registry.h,44475317,https://stackoverflow.com/q/44475317,https://raw.githubusercontent.com/tensorflow/t...
